In [1]:
import csv
import copy
import string
import random
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
#(algorithm will check incorrectness)
class Machine:
    def __init__(self, deck_dim, plate_dim, well_cap, tip_cap, head_pos = (0, 0), num_sol = 10): #deck_dim is a tuple (row_num, col_num)
        self.deck_dim = deck_dim #different from deck_row, deck_col defined later
        self.plate_dim = plate_dim
        self.well_cap = well_cap
        self.tip_cap = tip_cap
        self.head_pos = head_pos
        
        self.deck_row, self.deck_col = deck_dim[0] * plate_dim[0], deck_dim[1] * plate_dim[1]
        num_wells = self.deck_row * self.deck_col + plate_dim[0] * plate_dim[1] + 1 # Deck + Goal + Headtip
        self.status = np.zeros((num_sol, num_wells)) 
        
        self.read_goal()
        self.read_init()
        
    # PASSED
    def read_goal(self):
        with open('goal.txt', 'r') as f:
            lines = [[int(x) for x in line.strip().split(',')] for line in f.readlines()]
        self.goal_pos = tuple(lines[0])
        p = self.deck_row * self.deck_col
        for line in lines[1:]:
            x, y, sol, vol = line
            x_, y_ = sol, p + x * self.plate_dim[1] + y
            self.status[x_, y_] += vol
    # PASSED       
    def read_init(self):
        with open('init.txt', 'r') as f:
            lines = [[int(x) for x in line.strip().split(',')] for line in f.readlines()]
        self.init_pos = tuple(lines[0])
        p = self.init_pos[0] * self.plate_dim[0] * self.deck_col + self.init_pos[1] * self.plate_dim[1]
        for line in lines[1:]:
            x, y, sol, vol = line
            x_, y_ = sol, p + (x * self.deck_col + y)
            self.status[x_, y_] += vol
            
    # PASSED
    def simulate(self, protocol, mc = 1, zc = 1, ac = 1, dc = 1): # a list of cmds in format [op, x, y, vol] where x is solution id, y is position, v is volume
        head_pos = self.head_pos
        status = copy.deepcopy(self.status)
        cost = 0
        
        for command in protocol:
            c, status, head_pos = self.simulate_cmd(command, head_pos, status, mc, zc, ac, dc)
            cost += c
        return cost, status, head_pos
    
    # PASSED
    def simulate_cmd(self, command, head_pos, status, mc = 1, zc = 1, ac = 1, dc = 1):
        cost = 0
        op, x, y, vol = command
        
        to_r, to_c = y // self.deck_col, y % self.deck_col
        move_dst = abs(to_r - head_pos[0]) + abs(to_c - head_pos[1])
        cost += move_dst * mc
        
        if op == 0: #zero, move to y, tip -  (x, vol don't matter)
            cost += zc
            status[:, -1] = 0
        
        elif op == 1: #asp, move to y, well - , tip + 
            cost += ac
            status[x, y] -= vol
            status[x, -1] += vol
        
        elif op == 2: #dsp, move to y, well + , tip -
            cost += dc
            status[x, y] += vol
            status[x, -1] -= vol
       
        else:
            print("operation is not recoginized")
            return
        
        head_pos = (to_r, to_c)
        return cost, status, head_pos

In [3]:
deck_dim = (2, 3)
plate_dim = (2, 3)
well_cap = 1000
tip_cap = 100

In [4]:
lhpo = Machine(deck_dim, plate_dim, well_cap, tip_cap, num_sol = 3)